# 02 Local Ollama

Before running the notebook, make sure you have Ollama installed and running locally.

You can start it with: ollama serve

Then run the notebook.

In [8]:
import sys
from pathlib import Path
import requests
import json

# Add the project root to Python path so we can import src modules
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# Verify config file exists
config_file_path = project_root / "config" / "application.conf"
if not config_file_path.exists():
    raise FileNotFoundError(
        f"Required configuration file not found: {config_file_path}. "
        "Please create it before running this notebook."
    )

from src.config import AppConfig

# Load configuration
# Use the already calculated config_file_path
app_config = AppConfig(str(config_file_path))

print("Configuration loaded successfully!")
print(f"Embedding model: {app_config.embedding_llm.model_name}")
print(f"Generative model: {app_config.generative_llm.model_name}")

Configuration loaded successfully!
Embedding model: mxbai-embed-large:latest
Generative model: llama3.1:8b


In [9]:
# Check if Ollama is running locally
try:
    response = requests.get("http://127.0.0.1:11434/api/tags")
    if response.status_code == 200:
        print("Ollama is running on localhost:11434")
        models = response.json()["models"]
        print(f"Available models: {[model['name'] for model in models]}")
    else:
        print(f"Ollama responded with status: {response.status_code}")
except Exception as e:
    print(f"Error connecting to Ollama: {e}")
    print("Please make sure Ollama is installed and running locally.")
    print("You can start it with: ollama serve")

Ollama is running on localhost:11434
Available models: ['mxbai-embed-large:latest', 'llama3.3:latest', 'llama3.1:8b', 'nomic-embed-text:latest', 'mixtral:8x7b', 'mistral-small:24b', 'qwen2.5:14b']


In [10]:
# Pull the required models if they don't exist
def pull_model(model_name: str):
    """Pull a model if it doesn't exist"""
    try:
        # Check if model exists
        response = requests.get(f"http://127.0.0.1:11434/api/tags")
        if response.status_code == 200:
            models = response.json()["models"]
            model_names = [model['name'] for model in models]
            
            if model_name in model_names:
                print(f"Model {model_name} already exists")
                return True
            else:
                print(f"Pulling model {model_name}...")
                pull_response = requests.post(
                    "http://127.0.0.1:11434/api/pull",
                    headers={"Content-Type": "application/json"},
                    json={"name": model_name}
                )
                if pull_response.status_code == 200:
                    print(f"Model {model_name} pulled successfully")
                    return True
                else:
                    print(f"Failed to pull model {model_name}")
                    return False
    except Exception as e:
        print(f"Error pulling model {model_name}: {e}")
        return False

# Pull both models
embedding_ready = pull_model(app_config.embedding_llm.model_name)
generative_ready = pull_model(app_config.generative_llm.model_name)

Model mxbai-embed-large:latest already exists
Model llama3.1:8b already exists


In [11]:
# Test the models are working
if embedding_ready and generative_ready:
    print("All models are ready!")
    
    # Test embedding model
    print("\nTesting embedding model...")
    try:
        embedding_response = requests.post(
            "http://127.0.0.1:11434/api/embeddings",
            headers={"Content-Type": "application/json"},
            json={
                "model": app_config.embedding_llm.model_name,
                "prompt": "Hello, this is a test sentence."
            }
        )
        if embedding_response.status_code == 200:
            embedding_data = embedding_response.json()
            print(f"Embedding generated successfully!")
            print(f"Vector length: {len(embedding_data['embedding'])}")
        else:
            print(f"Embedding failed with status: {embedding_response.status_code}")
    except Exception as e:
        print(f"Error testing embedding: {e}")
    
    # Test generative model
    print("\nTesting generative model...")
    try:
        generation_response = requests.post(
            "http://127.0.0.1:11434/api/generate",
            headers={"Content-Type": "application/json"},
            json={
                "model": app_config.generative_llm.model_name,
                "prompt": "Say hello in a friendly way.",
                "stream": False
            }
        )
        if generation_response.status_code == 200:
            generation_data = generation_response.json()
            print(f"Generation successful!")
            print(f"Response: {generation_data['response']}")
        else:
            print(f"Generation failed with status: {generation_response.status_code}")
    except Exception as e:
        print(f"Error testing generation: {e}")
else:
    print("Some models are not ready. Please check the errors above.")

All models are ready!

Testing embedding model...
Embedding generated successfully!
Vector length: 1024

Testing generative model...
Generation successful!
Response: Hello there! It's so great to meet you! How's your day going so far?


In [12]:
# Example: Simple chat with the generative model
def chat_with_model(prompt: str, model_name: str = None):
    """Send a chat message to the generative model"""
    if model_name is None:
        model_name = app_config.generative_llm.model_name
    
    try:
        response = requests.post(
            "http://127.0.0.1:11434/api/generate",
            headers={"Content-Type": "application/json"},
            json={
                "model": model_name,
                "prompt": prompt,
                "stream": False
            }
        )
        
        if response.status_code == 200:
            data = response.json()
            return data['response']
        else:
            return f"Error: {response.status_code}"
    except Exception as e:
        return f"Error: {e}"

# Test chat
if generative_ready:
    print("Testing chat functionality...")
    response = chat_with_model("What is the capital of France?")
    print(f"Q: What is the capital of France?")
    print(f"A: {response}")

Testing chat functionality...
Q: What is the capital of France?
A: The capital of France is Paris.
